# ***Обучение первой модели по распознаванию дверей***

In [92]:
#Импортируем библиотеки
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import cv2
import os
import pytesseract
import proj_func as pf

#В ходе обработки видеофайлов будет использоваться Tesseract-OCR (Optical Char's Recognition)
#поэтому нужно явно указать директорию местонахождения его "движка":
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
config= '--oem 3 --psm 6'
WIDTH = 1920
HEIGHT = 1080

#Переопределяю размер видео для детектирования дверей
video_size = (224, 224, 3)

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
model = tf.keras.applications.MobileNetV2(input_shape = video_size,
                                          include_top = False,
                                          weights = 'imagenet')
model.trainable = False

#Загружаем все обучающие видео
video_path = 'train/video/'
data_path = 'train/data/'

#Открываем data-файл
data = pd.read_csv(os.path.join(data_path, 'fixed_train_data.csv'), delimiter=',')
dt = data['date']
dt_array = list()
dt.apply(lambda x: dt_array.append(x.split(' ')))
dt = dt_array

emp = data['employee']
emp_array = list()
emp.apply(lambda x: emp_array.append(str(x)))
emp = emp_array
emp = tf.keras.utils.to_categorical(emp, num_classes=12)

#Старт обучения по дверям


In [94]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = mobilenet_v2
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

class_names = np.array([str(x+1) for x in range(11)])
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_13 (KerasLayer)  (None, 1280)             2257984   
                                                                 
 dense_12 (Dense)            (None, 11)                14091     
                                                                 
Total params: 2,272,075
Trainable params: 14,091
Non-trainable params: 2,257,984
_________________________________________________________________


In [119]:
train_videos = list()

for video in os.listdir(video_path):
  train_videos.append(cv2.VideoCapture(os.path.join(video_path, video)))

FPS_CUT = 20
counter = 0

for video in train_videos:

  val_fr = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  val_tfr = val_fr//FPS_CUT
  print('Всего кадров:', val_fr)
  print('Считанных кадров:', val_tfr)
  while True:
    _, frame = video.read()
    frame = cv2.resize(frame, (WIDTH, HEIGHT))
    try:
      (x_d, y_d, w_d, h_d), (x_t, y_t, w_t, h_t) = pf.detect_pos_dt(config, frame)
      break
    except UnboundLocalError:
      continue

  while True:
    _, frame = video.read()
    frame = cv2.resize(frame, (WIDTH, HEIGHT))
    counter += 1
    if counter % FPS_CUT != 0:
      continue
    #Имеем точные геометрические рамки наших данных Дата\Время:
    thresh_time = frame[y_t - 4:y_t + h_t + 4, x_t:x_t + w_t]
    thresh_date = frame[y_d - 4:y_d + h_d + 4, x_d:x_d + w_d + 4]
    thresh_time = cv2.cvtColor(thresh_time, cv2.COLOR_RGB2GRAY)
    thresh_date = cv2.cvtColor(thresh_date, cv2.COLOR_RGB2GRAY)
    _, thresh_time = cv2.threshold(thresh_time, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    _, thresh_date = cv2.threshold(thresh_date, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh_time = cv2.blur(thresh_time, (5, 5))
    thresh_date = cv2.blur(thresh_date, (5, 5))
    date_now = pytesseract.image_to_string(thresh_date, config=config)
    time_now = pytesseract.image_to_string(thresh_time, config=config)
    dt = pf.normalize_dt(date_now, time_now)
    if dt in dt_array:
      index = dt_array.index(dt)
      y0 = emp[ind]
      x0 = list(cv2.resize(frame[160:][:], (224, 224))/255)
      print(x0)
      new_x0 = list()
      for i in x0:
        for j in i:
          for k in j:
            new_x0.append(k) 
      x0 = tf.ragged.constant([new_x0, y0])
      hist = model.fit(x0)
      print('Загружено кадров:', counter // FPS_CUT, '/', val_tfr)
    
    if cv2.waitKey(1) == ord('q'):
      cv2.destroyAllWindows()
      break
model.save_weights()

Выходные данные были обрезаны до нескольких последних строк (5000).
       [0.06666667, 0.12941176, 0.19215686],
       [0.23137255, 0.38039216, 0.54117647],
       [0.11372549, 0.3254902 , 0.47843137],
       [0.07058824, 0.09019608, 0.12941176],
       [0.0627451 , 0.0627451 , 0.05490196],
       [0.04313725, 0.04313725, 0.03529412],
       [0.03921569, 0.03921569, 0.03137255],
       [0.05882353, 0.05882353, 0.05098039],
       [0.0745098 , 0.0745098 , 0.06666667],
       [0.05490196, 0.0627451 , 0.05882353],
       [0.24313725, 0.37647059, 0.47843137],
       [0.08627451, 0.30980392, 0.49019608],
       [0.12941176, 0.31372549, 0.4627451 ],
       [0.        , 0.01176471, 0.        ],
       [0.20784314, 0.27058824, 0.2627451 ],
       [0.49803922, 0.55686275, 0.55686275],
       [0.83137255, 0.83529412, 0.81568627],
       [0.84313725, 0.82745098, 0.78823529],
       [0.89411765, 0.87843137, 0.83921569],
       [0.89803922, 0.88235294, 0.84313725],
       [0.89411765, 0.8745098 , 

ValueError: ignored

In [ ]:
import random
test_data = pd.read_csv('test/data/test.csv', delimiter=',')
test_data['employee'] = [random.randint(0,11) for _ in range(len(test_data))]
test_data['action'] = [random.randint(1,19) for _ in range(len(test_data))]
test_data.drop(columns = 'date', inplace = True)
test_data.to_csv('test/data/some/test.csv', index=False)
test_data

,id,employee,action
0,135,1,6
1,136,11,7
2,137,3,13
3,138,8,9
4,139,9,2
...,...,...,...
1116,1251,8,2
1117,1252,3,15
1118,1253,7,2
1119,1254,10,18
